In [1]:
import pymongo
import requests
import json
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
import datetime
from credentials import client_id,client_secret

In [2]:
def getLinkCounts(link):
    nexxt,last = link.split(",")
    url,rel = last.split(";")
    url = url.strip()
    url = url[1:-1]
    tmp,counts = url.split("&page=")
    return int(counts)        

In [3]:
url = "https://api.github.com/repos/tensorflow/tensorflow"
client_id = client_id
client_secret = client_secret

In [4]:
repo_response = requests.get(url,auth=(client_id, client_secret))

In [5]:
mongo_client = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
repodb = mongo_client["repo_collector"]
# print(myclient.list_database_names())
tensorflow_coll = repodb["tf_repo"]
# print(mydb.list_collection_names())

In [6]:
if repo_response.status_code == 200:
#     print(repo_response.json())
    repo_json = repo_response.json()  
    
    time = datetime.datetime.now()
    ISO_now = str(time.isoformat())
    contributors_count = 0
    orgmembers_count = 0
    pulls_count = 0
    
    contributors_url = "https://api.github.com/repos/tensorflow/tensorflow/contributors?per_page=1&anon=true"
    contributors_response = requests.get(contributors_url,auth=(client_id, client_secret))
    if contributors_response.status_code == 200:
        contributors_count = getLinkCounts(contributors_response.headers.get('Link'))
#         print(contributors_count)   
    
    members_url = "https://api.github.com/orgs/tensorflow/members?per_page=1"
    orgmembers_response = requests.get(members_url,auth=(client_id, client_secret))
    if orgmembers_response.status_code == 200:
        orgmembers_count = getLinkCounts(orgmembers_response.headers.get('Link'))
#         print(orgmembers_count)   
    
    pulls_url = "https://api.github.com/repos/tensorflow/tensorflow/pulls?state=open&per_page=1"
    pulls_response = requests.get(pulls_url,auth=(client_id, client_secret))
    if pulls_response.status_code == 200:
        pulls_count = getLinkCounts(pulls_response.headers.get('Link'))
#         print(pulls_count) 
    
    repo_add = {"contributors_count":contributors_count,"orgmembers_count":orgmembers_count,"open_pull_requests":pulls_count,"time_stamp":ISO_now}
    
    repo_json.update(repo_add)
#     print(repo_json)

In [7]:
tensorflow_coll.insert_one(repo_json)
print("Inserted "+str(datetime.date.today())+" tensorflow repo details")
mongo_client.close()

Inserted2021-07-16tensorflow repo details
